
**Created by:**

__[Viktor Varga](https://github.com/vvarga90)__

<br>

<img src="https://docs.google.com/uc?export=download&id=1WzgXsCoz8O-NeBlJTbuLPC1iIFDmgYt1" style="display:inline-block">
<hr>

# Python tutorial - Numpy, 1. fejezet

A Numpy a Python egyik legfontosabb csomagja: gyors numerikus számítások elvégzésére alkalmas. A Python nyelv alapértelmezett fordítóprogramja/értelmezője a CPython. Ez az implementáció összehasonlítva más nyelvekkel, nagyságrendekkel lassabb programokat eredményez. Ennek főbb okai a Python-t kényelmessé tevő dinamikus típusozásban, automatikus memóriakezelésben és abban keresendő, hogy a CPython szinte teljes mértékben értelmezőként (interpreterként) működik, a kód optimalizálására így nem igazán van lehetőség.

A sebességkülönbség akkor ütközik ki igazán, ha sok Python utasítást kell végrehajtani, például ha egy mátrixszorzást ciklusokkal és skalárok összeszorzásával valósítunk meg. A Numpy éppen ezért olyan függvényeket tartalmaz, melyek képesek ugyanazt a műveletet vektorosan, sokszor egymás után végrehajtani, anélkül, hogy Python ciklust kellene használnunk. A Numpy függvények hatékony nyelveken (pl. C, Fortran) írt kódokat hívnak meg, így a naiv Python megvalósításnál sokkal nagyobb hatékonyság érhető el Numpy használatával.

Az alábbi notebookban megtalálható egy futási idő összehasonlítás Numpy-t használó és nem használó algoritmusok közt:

https://drive.google.com/open?id=1jylusXuSJffdziUqw908wgeWXpSUxKbF

## Numpy alapok - `ndarray` típus

 A csomag legfontosabb típusa az `ndarray` n-dimenziós tömb implementáció. Hozzunk létre egy 2 dimenziós, 2x3 méretű, egész számokat tartalmazó tömböt. A tömb dimenziója nem keverendő a matematikában előforduló 'vektor dimenziószáma' fogalommal.

In [ ]:
import numpy as np  # import numpy package using the 'np' alias

a = np.array([[2,3,4],[1,5,8]], dtype=np.int32)   # create array
b = np.array([[3,2,9],[2,6,5]], dtype=np.int32)   # create array

print("The 'a' array:\n", a)
print("The 'b' array:\n", b)
print("Sum of the two arrays, elementwise:\n", a + b)
print("Multiplying all elements of 'a' by 2:\n", 2*a)
print("Adding up all elements of array 'a'. The sum is:", np.sum(a))

The 'a' array:
 [[2 3 4]
 [1 5 8]]
The 'b' array:
 [[3 2 9]
 [2 6 5]]
Sum of the two arrays, elementwise:
 [[ 5  5 13]
 [ 3 11 13]]
Multiplying all elements of 'a' by 2:
 [[ 4  6  8]
 [ 2 10 16]]
Adding up all elements of array 'a'. The sum is: 23


Ahogy látjuk, lehetőség van ugyanazon művelet elvégzésére az egész tömbön (vagy a tömb egy szeletén), anélkül, hogy ciklusokat kellene írnunk a kódba.

Az `ndarray` tömb a memóriában természetesen bájtok sorozataként, azaz egy 1 dimenziós vektorként kerül eltárolásra. Ha egyszer elkészítettünk egy ilyen tömböt, az elemeinek száma nem változhat meg: ha növelni akarjuk a tömb méretét, mindenképpen új tömböt kell létrehoznunk. A tömb alakja (shape) azonban változhat: egy 6 elemű tömb felvehet 2x3-as, 3x2-es, 6-os, 6x1-es, 6x1x1-es, 1x3x1x1x2x1-es, stb. alakot is. Ekkor a tömb elemeinek elhelyezkedése a memóriában változatlan marad. Azt a lehetőséget, hogy egy tömböt például 6 dimenzióban lássunk és indexelhessünk az `ndarray` típus speciális iterátorokkal valósítja meg. Például egy 1x3x1x1x2x1-es alakú tömböt indexelhetünk így: `my_array[0,2,0,0,1,0]`. A tömb megvalósítása ezt az indexelést fel tudja oldani úgy, hogy ismeri, az egyes tengelyek mentén történő 1-1 lépéshez mennyit kell lépni a memóriában. Ha a tömb alakját megváltoztatjuk, például 6x1-esre, akkor valójában csak az változik meg, hogy az egyes tengelyeken az index eggyel növeléséhez hány bájtnyi lépés szükséges a memóriában (`ndarray.strides`). Ennek részleteit egy külön, haladó szintű notebookban tárgyaljuk:

https://drive.google.com/open?id=1_eqSAFC-qUY-YTT8TFK2K_XSpL9ZGrjM

Nézzük tehát az `ndarray` típus attribútumait:

In [ ]:
a = np.array([[2,3,4],[1,5,8]], dtype=np.int32)

print("The shape of the array:", a.shape)   # a tuple
print("The data type of the array:", a.dtype)   # the type of the elements
print("The number of axes (dimensions of the array):", a.ndim)   # a.k.a length of the shape tuple
print("The size of the array (total number of elements):", a.size)   # the product of the shape
print("Length along the first axis:", a.shape[0])
print("Length along the first axis:", len(a))   # len() built-in python function

print("The strides of the array", a.strides)  # tells us how many bytes do we need to move in
                                              #   the memory to increase index along each axis


The shape of the array: (2, 3)
The data type of the array: int32
The number of axes (dimensions of the array): 2
The size of the array (total number of elements): 6
Length along the first axis: 2
Length along the first axis: 2
The strides of the array (12, 4)


## Tömbök alakjának megváltoztatása

Ahogy fentebb kifejtésre került, a tömbök alakja (shape) szabadon váltohat, feltéve, ha az új alakban az elemek száma nem változik. Ilyenkor az eredeti tömbről nem készül másolat, mindössze egy új nézet (view) készül a régi tömb felett. A nézet típusa is `ndarray`, tehát azt ugyancsak egy egyszerű tömbnek fogjuk látni. Fontos megérteni, hogy a nézet az eredeti tömb elemeit hivatkozza, azaz, ha az eredeti tömbelemek megváltoznak, a nézetben is a megváltozott elemeket fogjuk látni, illetve, ha a nézet tömb elemeit változtatjuk meg, az eredeti tömb is változik.

In [ ]:
a = np.arange(6, dtype=np.int32)
print("The 'a' array:\n", a)
print("   ... its shape is", a.shape)   # a tuple

b = a.reshape((2, 3))   # reshape to 2D shape: (2, 3)
print("\nThe 'a' array reshaped to 2D shape (2, 3):\n", b)
print("   ... its shape is", b.shape)

# We modify an item in the original array
a[0] = 42
print("\nThe modified 'a' array:\n", a)
print("The 2D 'b' view of the 'a' array is also modified:\n", b)

# Now we modify an item in the view array
b[1,1] = 99
print("\nThe content of the 'a' array after 2nd modification:\n", a)
print("The 'b' view after 2nd modification:", b)


The 'a' array:
 [0 1 2 3 4 5]
   ... its shape is (6,)

The 'a' array reshaped to 2D shape (2, 3):
 [[0 1 2]
 [3 4 5]]
   ... its shape is (2, 3)

The modified 'a' array:
 [42  1  2  3  4  5]
The 2D 'b' view of the 'a' array is also modified:
 [[42  1  2]
 [ 3  4  5]]

The content of the 'a' array after 2nd modification:
 [42  1  2  3 99  5]
The 'b' view after 2nd modification: [[42  1  2]
 [ 3 99  5]]


Mivel a tömb elemeinek száma fix, ezért az alak megváltoztatásánál az egyik tengely hosszát -1 -el lehet helyettesíteni, a megfelelő hossz ekkor automatikusan kiszámításra kerül.

In [ ]:
a = np.arange(6, dtype=np.int32).reshape((3, 2))    # ndarray.reshape()
print("The 'a' array:\n", a)
print("   ... its shape is", a.shape)

b = a.reshape(-1)   # reshape to 1D
print("\nThe 'b' array:\n", b)
print("   ... its shape is", b.shape)

c = b.reshape((1,-1,3,1))   # reshape to 4D: (1,2,3,1)
print("\nThe 'c' array:\n", c)
print("   ... its shape is", c.shape)

d = np.reshape(a, (-1,))   # np.reshape() works similarly to ndarray.reshape()
print("\nThe 'd' array:\n", d)
print("   ... its shape is", d.shape)


The 'a' array:
 [[0 1]
 [2 3]
 [4 5]]
   ... its shape is (3, 2)

The 'b' array:
 [0 1 2 3 4 5]
   ... its shape is (6,)

The 'c' array:
 [[[[0]
   [1]
   [2]]

  [[3]
   [4]
   [5]]]]
   ... its shape is (1, 2, 3, 1)

The 'd' array:
 [0 1 2 3 4 5]
   ... its shape is (6,)


Tömbök tengelyeinek sorrendje is felcserélhető (`np.swapaxes()`, `np.transpose()`, `ndarray.T`). Mátrixok transzponálásának megvalósításához is ezeket a műveleteket érdemes használni.

## Tömbök létrehozása

Az `ndarray` tömbök elemeinek típusát a tömbök létrehozásakor kell megadni. A tömb minden elemének típusa azonos lesz. Ez alól kivételt képeznek a ritkán használt strukturált tömbök (structured arrays) és rekord tömbök (recarray), ezekkel azonban nem fogunk foglalkozni.

Dokumentáció:
https://docs.scipy.org/doc/numpy/reference/routines.array-creation.html

In [ ]:
a = np.zeros((2,3), dtype=np.float32)
print("New floating point array, filled with zeros:\n", a)

b = np.ones((2,2), dtype=np.int32)
print("New integer array, filled with ones:\n", b)

c = np.empty((1,3), dtype=np.float32)  # NO INITIALIZATION of elements, values are undefined
print("New EMPTY (uninitialized) array (elements may have any starting value):\n", c)

d = np.array([[1.,.2,-2.8],[2.6,3.2,-.2]], dtype=np.float32)
print("New floating point array, elements are given explicitly:\n", d)
print("    ... it's shape is ", d.shape)

New floating point array, filled with zeros:
 [[0. 0. 0.]
 [0. 0. 0.]]
New integer array, filled with ones:
 [[1 1]
 [1 1]]
New EMPTY (uninitialized) array (elements may have any starting value):
 [[3.8960277e-37 0.0000000e+00 2.8025969e-45]]
New floating point array, elements are given explicitly:
 [[ 1.   0.2 -2.8]
 [ 2.6  3.2 -0.2]]
    ... it's shape is  (2, 3)


Főbb adattípusok (`dtype`):
* `np.bool_`: Logikai adattípus (True vagy False). A True 1-nek, a False 0-nak felel meg.
* `np.int8, int16, int32, int64`: 1, 2, 4, 8 bájtos egész számokat tartalmazó adattípus, pl. int8 értéke [-128, 127] intervallumból van.
* `np.uint8, uint16, uint32, uint64`: 1, 2, 4, 8 bájtos nemnegatív egész számokat tartalmazó adattípus, pl. uint8 értéke [0, 255] intervallumból van.
* `np.float16, float32, float64`: 2, 4, 8 bájtos valós számokat tartalmazó adattípus. A mai videokártyák többsége a float32 (single precision) adattípus használatára optimalizált.

Tömbök létrehozása egyenlő lépésközű sorozatokból:

In [ ]:
# ranges, interavals

a = np.arange(5, dtype=np.int64)   # end value is given
print("Array with elements from range(5): ", a)
print("    ... shape of the array is:", a.shape)

b = np.arange(5, 10, dtype=np.float16)   # start & end value is given
print("\nArray with elements from range(5, 10): ", b)

c = np.arange(10, -4, -2, dtype=np.float64)   # start, end & step value is given
print("\nArray with elements from range(10, 4, -2): ", c)

b2 = np.arange(10, 5, dtype=np.float16)   # start & end value is given
print("\nArray with elements from (empty) range(10, 5): ", b2)
print("    ... shape of the empty array is:", b2.shape)

d = np.linspace(10., 13., num=7, endpoint=False, dtype=np.float64)
print("\nArray with equally spaced numbers from interval:\n", d)


Array with elements from range(5):  [0 1 2 3 4]
    ... shape of the array is: (5,)

Array with elements from range(5, 10):  [5. 6. 7. 8. 9.]

Array with elements from range(10, 4, -2):  [10.  8.  6.  4.  2.  0. -2.]

Array with elements from (empty) range(10, 5):  []
    ... shape of the empty array is: (0,)

Array with equally spaced numbers from interval:
 [10.         10.42857143 10.85714286 11.28571429 11.71428571 12.14285714
 12.57142857]


Véletlen számok generálása:

In [ ]:
# random numbers
r1 = np.random.rand(1, 4, 1)
print("(1,4,1) shaped array with random elements from [0,1) interval (uniform):\n ", r1)
print("    ... shape of the array:", r1.shape, "; datatype of the array:", r1.dtype)

r2 = np.random.randn(2)
print("\n(2,) shaped array with random elements from standard normal (Gaussian) distribution:\n ", r2)

r3 = np.random.randint(5, size=(2,5), dtype=np.uint16)
print("\n(2,5) shaped array with random integer elements from [0,4] interval (uniform):\n ", r3)


(1,4,1) shaped array with random elements from [0,1) interval (uniform):
  [[[0.06135587]
  [0.64579058]
  [0.27477774]
  [0.25016094]]]
    ... shape of the array: (1, 4, 1) ; datatype of the array: float64

(2,) shaped array with random elements from standard normal (Gaussian) distribution:
  [-0.77897088 -0.18866774]

(2,5) shaped array with random integer elements from [0,4] interval (uniform):
  [[2 4 4 1 0]
 [2 2 0 0 2]]


## Konverzió adattípusok közt

Szükség lesz később arra, hogy egy adott adattípusú tömböt más adattípusúra konvertáljunk. Ilyenkor új tömb készül.

Egy gyakori use case: képeket, videókat sokszor uint8 formátumban tárolnak (egy pixel fényereje színcsatornánként 0 és 255 közti értékekkel van reprezentálva), de a neuronhálókban valós számokkal szeretnénk számolni, hiszen a gradiens és a rejtett reprezentációk nem feltétlenül egész számok, ezért float32 formátumra konvertáljuk őket.

A konverzió során természetesen figyelni kell az  információvesztés lehetőségére, ami adódhat a pontosság csökkentéséből, vagy alul-/túlcsordulásból (under/overflow).

In [ ]:
a = np.array([[2,3,125], [255, 0, 45]], dtype=np.uint8)
print("Data type of the 'a' array:", a.dtype)
print("The 'a' array:\n", a)

b = a.astype(np.float32)  # type conversion
print("\nData type of the 'b' array:", b.dtype)
print("The 'b' array:\n", b)


Data type of the 'a' array: uint8
The 'a' array:
 [[  2   3 125]
 [255   0  45]]

Data type of the 'b' array: float32
The 'b' array:
 [[  2.   3. 125.]
 [255.   0.  45.]]


## Basic indexing, tömb nézetek

Egy Numpy tömbnek egyszerre több eleme indexelhető, így ciklus nélkül tudjuk több elemen ugyanazt a műveletet elvégezni.

Kétfajta indexelési módszert különböztetünk meg: a **basic** és az **advanced indexing**-et.

A **basic indexing** során a tömbök elemeit a Python listákhoz hasonló módon, range-el tudjuk indexelni, de akár több dimenzió mentén egyszerre. Például az `a` 1 dimenziós, legalább 10 elemű tömb esetén `a[5:10:2]` az 5, 7, 9-es indexű elemeket fogja hivatkozni. `a[5:10:2]` maga is ndarray típusú lesz, a fent említett három elemmel. Az ehhez hasonló **basic indexing** során készült tömb azonban csak egy _nézete_ (view) lesz az eredeti tömbnek, tehát, ha az eredeti tömb ezen elemei változnak, a nézet elemei is változni fognak.

Az **advanced indexing**ről a következő notebookban lesz szó.


Egyetlen elem elérése és írása:

In [ ]:
a = np.arange(6, dtype=np.int32).reshape((3, 2))
print("The 'a' array:\n", a)
print("   ... its shape is", a.shape)

print("\nAccessing a single element:", a[0,1])

a[0,1] = 42  # writing a single element in the array
print("\nThe 'a' array:\n", a)

The 'a' array:
 [[0 1]
 [2 3]
 [4 5]]
   ... its shape is (3, 2)

Accessing a single element: 1

The 'a' array:
 [[ 0 42]
 [ 2  3]
 [ 4  5]]


Negatív index (hátulról számolva indexelés):

In [ ]:
# indexing with negative numbers: -1: last, -2: last but one

print("\nAccessing a single element:", a[0,-1])
print("Accessing a single element:", a[-2,-1])


Accessing a single element: 42
Accessing a single element: 3


Range indexelés (a Python listákhoz hasonló szintaxis):

In [ ]:
a = np.arange(6, dtype=np.int32)
print("The 'a' array:\n", a)
print("   ... its shape is", a.shape)

print("\nA slice of array 'a':", a[2:4])  # syntax -> start:stop:step_size

print("Another slice of array 'a':", a[2:])   # from #2 till end

print("A third slice of array 'a':", a[:4])   # from start till #4 (exclusive)

print("\nEvery second item in array 'a':", a[::2])

print("Reversed 'a':", a[::-1])

print("Custom slice 'a':", a[4:0:-2])

The 'a' array:
 [0 1 2 3 4 5]
   ... its shape is (6,)

A slice of array 'a': [2 3]
Another slice of array 'a': [2 3 4 5]
A third slice of array 'a': [0 1 2 3]

Every second item in array 'a': [0 2 4]
Reversed 'a': [5 4 3 2 1 0]
Custom slice 'a': [4 2]


Range indexelés egyszerre több tengelyen:

In [ ]:
a = np.arange(30, dtype=np.int32).reshape((5,6))
print("The 2D 'a' array:\n", a)
print("   ... its shape is", a.shape)

print("\nSingle index along axis#0, entire range along#1: ", a[3,:])  # 4th row

print("... same as previous, last axis indexing omitted: ", a[3])  # 4th row

print("Single index along axis#0, custom range along#1: ", a[3,6:0:-2])

print("\nEntire range along axis#0, single index along#1: ", a[:,3])  # 4th column

print("... same as previous, all axes omitted except last one: ", a[..., 3])  # 4th row

print("\nCustom range along both axes:\n", a[::3,6:0:-2])

The 2D 'a' array:
 [[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]
 [24 25 26 27 28 29]]
   ... its shape is (5, 6)

Single index along axis#0, entire range along#1:  [18 19 20 21 22 23]
... same as previous, last axis indexing omitted:  [18 19 20 21 22 23]
Single index along axis#0, custom range along#1:  [23 21 19]

Entire range along axis#0, single index along#1:  [ 3  9 15 21 27]
... same as previous, all axes omitted except last one:  [ 3  9 15 21 27]

Custom range along both axes:
 [[ 5  3  1]
 [23 21 19]]


Üres range. Megengedett, hogy egy tömb mérete 0 legyen, ekkor alakja legalább az egyik tengely mentén 0...

In [ ]:
b = a[8:,:]
print("\nAn empty slice of 'a': ", b)  # no error if RANGE is out of bounds
print("   ... its shape is", b.shape)

# b = a[8,:]   # would result in an error, because axis#0 SINGLE index is out of bounds


An empty slice of 'a':  []
   ... its shape is (0, 6)


Tömbök szeleteinek írása:

In [ ]:
a = np.arange(12, dtype=np.int32).reshape((4,3))
print("The 2D 'a' array:\n", a)
print("   ... its shape is", a.shape)

a[::2,1:3] = 42
print("\nThe modified 2D 'a' array:\n", a)

b = a[:,:2]   # 'b' is a view of array 'a', if we write 'b', we also write 'a'
print("\nThe 'b' array (a view of 'a'):\n", b)
print("   ... its shape is", b.shape)

b[:] = 99   # with [:] (or [:,:], ...) we refer to all elements in 'b'
print("\nThe 'b' array (a view of 'a'):\n", b)
print("\nThe 'a' array:\n", a)


The 2D 'a' array:
 [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]
   ... its shape is (4, 3)

The modified 2D 'a' array:
 [[ 0 42 42]
 [ 3  4  5]
 [ 6 42 42]
 [ 9 10 11]]

The 'b' array (a view of 'a'):
 [[ 0 42]
 [ 3  4]
 [ 6 42]
 [ 9 10]]
   ... its shape is (4, 2)

The 'b' array (a view of 'a'):
 [[99 99]
 [99 99]
 [99 99]
 [99 99]]

The 'a' array:
 [[99 99 42]
 [99 99  5]
 [99 99 42]
 [99 99 11]]


## Egyszerű aritmetikai műveletek, tömbök alakjának egyeztetése (broadcasting)

Egyszerű elemenkénti (elementwise) aritmetika azonos méretű tömbökön. Két tömb elemeinek összeadása az alábbi módon történik. Nincs szükség Python ciklus írására.

In [ ]:
a = np.arange(6, dtype=np.int32).reshape((2,3))
print("The 2D 'a' array:\n", a)
print("   ... its shape is", a.shape)

b = np.ones_like(a, dtype=np.int32)
print("\nThe 2D 'b' array:\n", b)
print("   ... its shape is", b.shape)

# now we add the two arrays

c = a+b
print("\nThe result array:\n", c)

The 2D 'a' array:
 [[0 1 2]
 [3 4 5]]
   ... its shape is (2, 3)

The 2D 'b' array:
 [[1 1 1]
 [1 1 1]]
   ... its shape is (2, 3)

The result array:
 [[1 2 3]
 [4 5 6]]


Eddig minden világos, azonos méretű tömböket össze tudunk adni, szorozni, stb. elemenként. Azonban a Numpy lehetővé teszi bizonyos szabályok szerint, hogy eltérő méretű tömbökön is végezhetők legyen elemenkénti műveletek. Ehhez a Numpy automatikusan egyezteti a tömbök méretét, szükség esetén az elemek megismétlésével bizonyos tengelyek mentén. Ez a **broadcasting**.

Mi történik pontosan, ha egy konstanssal meg akarjuk növelni egy tömb minden elemét?

In [ ]:
a = np.arange(6, dtype=np.int32).reshape((2,3))
print("The 2D 'a' array:\n", a)
print("   ... its shape is", a.shape)

a = a+1
print("\nThe modified 'a' array:\n", a)


The 2D 'a' array:
 [[0 1 2]
 [3 4 5]]
   ... its shape is (2, 3)

The modified 'a' array:
 [[1 2 3]
 [4 5 6]]


Ilyenkor, a Numpy először becsomagolja a Python int típusú '1' literált egy (1,) alakú Numpy tömbbe, melyben az egyetlen elem az 1 lesz. Ezután az ábrán látható módon az elemek ismételgetésével mindkét tengely mentén megnöveli a méretét (2,3) alakra, hogy alakja pontosan egyezzen az 'a' tömb alakjával.

![alt text](https://drive.google.com/uc?export=download&id=1O_R6m1Jmt2ffset2D7GtV73YRgW9yYvc)

Broadcasting esetén **a kiterjesztett méretű tömb valójában csak egy nézet (view) az eredeti tömb felett**, nem foglalódik le új tömb a memóriában. Az elemek ismétlődése csak látszólagos: ilyenkor ugyanaz a memóriaterület kerül elérésre. Ennek múködésének részleteiről az alábbi notebookban lehet olvasni:

https://drive.google.com/open?id=1_eqSAFC-qUY-YTT8TFK2K_XSpL9ZGrjM

Tekintsük a broadcasting további eseteit. Értelmezhetjük úgy a 2 dimenziós tömböt, mint egy mátrixot. Adjunk hozzá egy vektort a mátrix minden sorához!

In [ ]:
a = np.arange(6, dtype=np.int32).reshape((2,3))
print("The 2D 'a' array:\n", a)
print("   ... its shape is", a.shape)

vec = np.arange(10,40,10, dtype=np.int32)
print("\nThe 1D 'vec' array:\n", vec)
print("   ... its shape is", vec.shape)

a = a+vec
print("\nThe modified 'a' array:\n", a)

The 2D 'a' array:
 [[0 1 2]
 [3 4 5]]
   ... its shape is (2, 3)

The 1D 'vec' array:
 [10 20 30]
   ... its shape is (3,)

The modified 'a' array:
 [[10 21 32]
 [13 24 35]]


Ekkor, a (3,) alakú vektor egy új tengelyt kap: (1,3) alakra lesz hozva, majd a #0 indexű tengely mentén elemismétlésekkel (2,3) alakot kap.

![alt text](https://drive.google.com/uc?export=download&id=1hCbOpGPvU84B6Jg5fpBT866w49VYcamu)

Próbáljunk most egy oszlopvektort hozzáadni a mátrixhoz!

In [ ]:
a = np.arange(6, dtype=np.int32).reshape((2,3))
print("The 2D 'a' array:\n", a)
print("   ... its shape is", a.shape)

vec = np.arange(10,30,10, dtype=np.int32)
print("\nThe 1D 'vec' array:\n", vec)
print("   ... its shape is", vec.shape)

# a = a+vec    # ValueError: operands could not be broadcast together with shapes (2,3) (2,)

The 2D 'a' array:
 [[0 1 2]
 [3 4 5]]
   ... its shape is (2, 3)

The 1D 'vec' array:
 [10 20]
   ... its shape is (2,)


A Numpy hibát dobna, nem tudja végrehajtani a műveletet. Miért?

A Numpy a broadcasting során a két tömb tengelyeinek hosszát hátulról kezdve próbálja meg egyeztetni. Az előbbi példában ez egyezett, azonban itt a mátrix utolsó tengelyének 3-as hossza és a vektor 2-es hossza nem kompatibilis egymással.

![alt text](https://drive.google.com/uc?export=download&id=1ht8GtbplkKRQEQEcpFz4iCiJfg8bdCn8)

Mit tehetünk?

Tudomására kell hozni a Numpy-nak, hogy a vektort a mátrixnak nem az #1-es, hanem a #0-ás indexű tengelyével szeretnénk egyeztetni. Mivel a tengelyek egyeztetése hátulról kezdve történik, ezért a vektort 2 dimenzióssá kell alakítanunk kézzel, egy extra (1 hosszú) utolsó tengely hozzáadásával. Azaz, a (2,) alakú vektort (2, 1) alakra hozzuk kézzel és ezután próbáljuk elvégezni az összeadást.

In [ ]:
a = np.arange(6, dtype=np.int32).reshape((2,3))
print("The 2D 'a' array:\n", a)
print("   ... its shape is", a.shape)

vec = np.arange(10,30,10, dtype=np.int32)
print("\nThe 1D 'vec' array:\n", vec)
print("   ... its shape is", vec.shape)

a = a + vec.reshape((2, 1))
print("\nThe modified 'a' array:\n", a)

The 2D 'a' array:
 [[0 1 2]
 [3 4 5]]
   ... its shape is (2, 3)

The 1D 'vec' array:
 [10 20]
   ... its shape is (2,)

The modified 'a' array:
 [[10 11 12]
 [23 24 25]]


A művelet sikeres volt, a következő történt: a (2,1) alakú 2 dimenziósra alakított vektor az #1 indexű tengely mentén megismétlődött háromszor. Ez természetesen ugyancsak egy nézet, valójában csak az iteráció változott meg a vektoron, továbbra is 2 elemnyi helyet foglal a memóriában.

![alt text](https://drive.google.com/uc?export=download&id=1G-cPJzXbpzXoHXUJwJ-lxk4LA8PsyI7d)

Broadcasting segítségével szorzótábla is készíthető:

In [ ]:
v1 = np.arange(3, dtype=np.int32)
print("The 'v1' vector:\n", v1)
print("   ... its shape is", v1.shape)

v2 = (np.arange(3, dtype=np.int32)+3).reshape(3,1)
print("The 'v2' vector:\n", v2)
print("   ... its shape is", v2.shape)

r = v1 * v2
print("\nThe result array:\n", r)

The 'v1' vector:
 [0 1 2]
   ... its shape is (3,)
The 'v2' vector:
 [[3]
 [4]
 [5]]
   ... its shape is (3, 1)

The result array:
 [[ 0  3  6]
 [ 0  4  8]
 [ 0  5 10]]


Ebben az esetben mindkét tömb alakja változott az egyeztetéshez. Az első (3,) alakú vektor új #0 indexű tengelyt kapott, hogy a dimenzióinak száma egyezzen a második vektorral, így alakja (1,3) lett. Ekkor, az első vektor 3 hosszú tengelyét a második vektor 1 hosszú tengelyével, míg a második vektor 3 hosszú tengelyét az első vektor új tengelyével egyezteti a Numpy, így mindkét tengely mentén az 1 hossz kiterjesztődik látszólagos ismétléssel 3 hosszúra. Végül tehát két (3,3) alakú mátrix elemei szorzódnak össze páronként, ami a végső eredményt adja.

![alt text](https://drive.google.com/uc?export=download&id=1LYRdTn-99cFHtCZD3teSNP8bK-zWFG8b)

**Melyek tehát a broadcasting pontos szabályai?**

- Ha a két tömb dimenzióinak száma nem egyezik, a kevesebb dimenzióval rendelkező tömb új (1 hosszú) első tengelyeket kap. Éppen annyit, hogy dimenzióik száma egyezzen.
- Ezután a két tömb tengelyei páronként egyeztetésre kerülnek: az első tömb #i indexű tengelyének hossza kompatibilis kell, hogy legyen a másik tömb #i indexű tengelyének hosszával. Két tengely kompatibilis, ha hosszuk egyezik, vagy bármelyiknek hossza 1. Ha hosszuk egyezik, az adott tengely mentén nincs szükség ismétlésre. Ha hosszuk nem egyezik és az egyik tömb esetében a tengely hossza 1, akkor az adott tengely mentén a teljes tömb megismétlődik, annyiszor, hogy a tengely hossza egyezzen a másik tömb tengelyhosszával.
- Ha két tömb bármelyik tengely mentén nem egyeztetehető, akkor a Numpy hibát dob, a broadcasting nem végezhető el.

Dokumentáció: https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html

**A `None` index**

Ha új (1 hosszú) tengelyt szeretnénk hozzáadni a tömb alakjához, a `None` (vagy `np.newaxis`) index használatával is megtehetjük azt:

In [ ]:
# the previous code block in a more compact way

r = np.arange(3, dtype=np.int32) * (np.arange(3, dtype=np.int32)+3)[:,None]
print("\nThe result array:\n", r)


The result array:
 [[ 0  3  6]
 [ 0  4  8]
 [ 0  5 10]]


**Néhány fontosabb elemenkénti (elementwise) művelet**

In [ ]:
a = np.arange(6, dtype=np.int32).reshape((2,3))
print("The 2D 'a' array:\n", a)
print("   ... its shape is", a.shape)

vec = np.array([2,4], dtype=np.int32)[:,None]
print("\nThe 'vec' array:\n", vec)
print("   ... its shape is", vec.shape)

#
print("\nAdding:\n", a+vec)
print("\nSubtracting:\n", a-vec)
print("\nMultiplying:\n", a*vec)
print("\nFloat division:\n", a/vec)   # this would be integer div in Python2 since both array are of int type!!!
print("\nInteger division:\n", a//vec)
print("\nPower:\n", a**vec)
print("\nModulo:\n", a%vec)
print("\nElementwise maximum:\n", np.maximum(a, vec))
print("\nEqual:\n", a == vec)
print("\nGreater:\n", a > vec)

The 2D 'a' array:
 [[0 1 2]
 [3 4 5]]
   ... its shape is (2, 3)

The 'vec' array:
 [[2]
 [4]]
   ... its shape is (2, 1)

Adding:
 [[2 3 4]
 [7 8 9]]

Subtracting:
 [[-2 -1  0]
 [-1  0  1]]

Multiplying:
 [[ 0  2  4]
 [12 16 20]]

Float division:
 [[0.   0.5  1.  ]
 [0.75 1.   1.25]]

Integer division:
 [[0 0 1]
 [0 1 1]]

Power:
 [[  0   1   4]
 [ 81 256 625]]

Modulo:
 [[0 1 0]
 [3 0 1]]

Elementwise maximum:
 [[2 2 2]
 [4 4 5]]

Equal:
 [[False False  True]
 [False  True False]]

Greater:
 [[False False False]
 [False False  True]]


**Elemenkénti műveletek logikai (bool) típusú tömbökön:**

In [ ]:
a = np.arange(10, dtype=np.int32).reshape((2,5))
b = (a % 3 == 0)    # True for elements divisible by 3
print("The 2D 'b' array:\n", b)
print("   ... its shape is", b.shape)
print("   ... its data type is", b.dtype)  # np.bool_

c = (a % 2 != 0)    # True for elements not divisible by 2 (odd numbers)
print("\nThe 2D 'c' array:\n", c)
print("   ... its shape is", c.shape)
print("   ... its data type is", c.dtype)  # np.bool_

print("\nElementwise logical AND of the two arrays:\n", b & c)
print("\nElementwise logical OR of the two arrays:\n", b | c)
print("\nElementwise logical NOT of array 'b':\n", ~b)


The 2D 'b' array:
 [[ True False False  True False]
 [False  True False False  True]]
   ... its shape is (2, 5)
   ... its data type is bool

The 2D 'c' array:
 [[False  True False  True False]
 [ True False  True False  True]]
   ... its shape is (2, 5)
   ... its data type is bool

Elementwise logical AND of the two arrays:
 [[False False False  True False]
 [False False False False  True]]

Elementwise logical OR of the two arrays:
 [[ True  True False  True False]
 [ True  True  True False  True]]

Elementwise logical NOT of array 'b':
 [[False  True  True False  True]
 [ True False  True  True False]]
